In [ ]:
import tensorflow as tf
import os
import numpy as np
from PIL import Image
import random
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
from imblearn.over_sampling import SMOTE
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras import models
from keras import layers
import tensorflow_addons as tfa

!pip install visualkeras
import visualkeras

tfk = tf.keras
tfkl = tf.keras.layers

# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Dataset folders 
dataset_dir = '../input/dataset21/dataset1/training_set1'
validation_dir = '../input/dataset21/dataset1/validation_set1'

input_shape = (256, 256, 3)
epochs = 50

In [ ]:
#CREATE THE MODEL WITH VGG16

# Plot example images from dataset
labels = ['Apple','Blueberry','Cherry','Corn','Grape','Orange','Peach','Pepper','Potato','Raspberry','Soybean','Squash','Strawberry','Tomato']


# Download the VGG16 model
supernet = tfk.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=(256,256,3))

supernet.trainable = False


inputs = tfk.Input(shape=(256,256,3))  
#FIRST GAUSSIAN NOISE LAYER 
inputs1 = tf.keras.layers.GaussianNoise(0.1)(inputs)
#RESIZING
x = tfkl.Resizing(256, 256, interpolation="bicubic")(inputs1)
#SUPERNET VGG16
x1 = supernet(x)
#BATCHNORM
batchNorm5 = tfkl.BatchNormalization(axis=-1)(x1)
#GLOBAL AVERAGE POOLING LAYER
glob_av_pooling = tfkl.GlobalAveragePooling2D(name='GloablPooling')(batchNorm5) 
#SECOND GAUSSIAN NOISE LAYER
gauss = tf.keras.layers.GaussianNoise(0.1)(glob_av_pooling)
#DROPOUT LAYER  AND DENSE FINAL LAYER 
classifier_layer2 = tfkl.Dropout(0.3, seed=seed)(gauss)
outputs = tfkl.Dense(units=14, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed), name='Output')(classifier_layer2)



# Connect input and output through the Model class
model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model with adaptive learning rate starting equal to 0.1 
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=0.1), metrics='accuracy')
model.summary()

#visualize the model
visualkeras.layered_view(model, legend=True, spacing=20, scale_xy=2)

In [ ]:
#MODIFY VGG16 FREEZING SOME LAYERS:  FINE TUNING

# Set all VGG layers to True
model.get_layer('vgg16').trainable = True
for i, layer in enumerate(model.get_layer('vgg16').layers):
       print(i, layer.name, layer.trainable)

# Freeze first N layers, e.g., until 10th
for i, layer in enumerate(model.get_layer('vgg16').layers[:11]):
      layer.trainable=False
for i, layer in enumerate(model.get_layer('vgg16').layers):
       print(i, layer.name, layer.trainable)
model.summary()

In [ ]:
#AUGMENTATION 


#Augmentation on training

# Create an instance of ImageDataGenerator with Data Augmentation and data preprocessing
aug_train_data_gen = ImageDataGenerator(rotation_range=30,
                         height_shift_range=0.5,
                         width_shift_range=0.5,
                         zoom_range=0.4,
                         horizontal_flip=True,
                         vertical_flip=True,
                         fill_mode='reflect',
                         preprocessing_function= preprocess_input)
                        

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
aug_train_gen = aug_train_data_gen.flow_from_directory(directory=dataset_dir,
                                               target_size=(256,256),
                                               color_mode='rgb',
                                               classes=None, 
                                               batch_size=32,
                                               shuffle=True,
                                               seed=seed)
                                               
#validation

# Create an instance of ImageDataGenerator with only preprocessing
val_data_gen = ImageDataGenerator( preprocessing_function=preprocess_input)

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
val_gen = val_data_gen.flow_from_directory(directory=validation_dir,
                                               target_size=(256,256),
                                               color_mode='rgb',
                                               classes=None, 
                                               batch_size=32,
                                               shuffle=False,
                                               seed=seed)
                                               


In [ ]:
#Class weights due to unbalanced dataset

class_weights = class_weight.compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(aug_train_gen.classes),
                                        y = aug_train_gen.classes   )                                                 
                                    
class_weights = dict(zip(np.unique(aug_train_gen.classes), class_weights))

print(class_weights)



In [ ]:
from datetime import datetime
def create_callbacks():
    callbacks = []

    exps_dir = os.path.join('trained')
    if not os.path.exists(exps_dir):
              os.makedirs(exps_dir)

    now = datetime.now().strftime('%b%d_%H-%M-%S')

    exp_dir = os.path.join(exps_dir, 'ToyNotebook' + '_' + str(now))
    if not os.path.exists(exp_dir):
                  os.makedirs(exp_dir)

   # Model checkpoint
   # --------------------------------------------------------------------
    ckpt_dir = os.path.join(exp_dir, 'trained/checkpoint.ckpt')
    if not os.path.exists(ckpt_dir):
                  os.makedirs(ckpt_dir)

    ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp'),
                                                             save_weights_only=False,
                                                             save_best_only=True)
    callbacks.append(ckpt_callback)

    # Early Stopping
    # ------------------------------------------------------------------
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True)
    callbacks.append(es_callback)
    
    # Dynamically adjust Learning Rate
    # ------------------------------------------------------------------
    lr_callbacks = tf.keras.callbacks.ReduceLROnPlateau( monitor='val_loss' ,factor=0.1, patience=5,verbose=1,min_lr=1e-4) 
    callbacks.append(lr_callbacks)
    
    return callbacks




In [ ]:
tf.get_logger().setLevel('WARNING') 

# Create folders and callbacks and fit
callbacks = create_callbacks()

# Train the model
history = model.fit(
    x = aug_train_gen,
    epochs = epochs,
    validation_data = val_gen,
    class_weight = class_weights,
    callbacks = callbacks)

In [ ]:
#PLOTS FOR MODEL EVALUATION:

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(loss))

plt.figure(figsize=(32, 16))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

y_pred = np.argmax(model.predict_generator(val_gen, steps=len(val_gen)), axis=1) 
y_true = np.concatenate([np.argmax(val_gen[i][1], axis=1) for i in range(len(val_gen))])

# Compute the classification metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
print('Accuracy:',accuracy.round(4))
print('Precision:',precision.round(4))
print('Recall:',recall.round(4))
print('F1:',f1.round(4))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm.T,
            xticklabels=labels,
            yticklabels=labels,
            annot=True, 
            fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.show()


In [ ]:
#SAVE THE MODEL

model.save("Sub6_1")